# DataFrames

* a `DataFrame` is a popular data structure (in many languages) for storing data in tabular form (rows, columns)
* the Julia package `DataFrames.jl` has many functions that operate on `DataFrames` that allow us to quickly query and manipulate data
* it can handle missing values efficiently
* the Julia package `CSV.jl` allows us to quickly and flexibly read/write CSV (= comma-separated value) files from/to `DataFrame`s

You need to install these packages:
```julia
using Pkg
Pkg.add("DataFrames")
Pkg.add("CSV")
```
or, for pros, open a Julia [REPL](https://en.wikipedia.org/wiki/Read–eval–print_loop), hit the `]` key to go into package mode, then `add DataFrames CSV`.

* `DataFrames.jl` [documentation](http://juliadata.github.io/DataFrames.jl/stable/)
* `CSV.jl` [documentation](https://juliadata.github.io/CSV.jl/stable/)

In [57]:
# always list packages you use at the top
using DataFrames # load into namespace all of the functions, data types, variables from the `DataFrames` package
using CSV
using Printf # for print formating

## construct a `DataFrame`

In [45]:
city = ["Corvallis", "Portland", "Eugene"]
population = [57961, 647805, 168916]

df_cities = DataFrame(city=city, population=population)

# this works too of course:
#   df_cities = DataFrame(city=["Corvallis", "Portland", "Eugene"], population=[57961, 647805, 168916])

,city,population
,String,Int64
1,Corvallis,57961
2,Portland,647805
3,Eugene,168916


## add rows to a `DataFrame`

array-style (must memorize order)

In [46]:
push!(df_cities, ["Bend", 94520])
df_cities

,city,population
,String,Int64
1,Corvallis,57961
2,Portland,647805
3,Eugene,168916
4,Bend,94520


dictionary style (no need to memorize order)

In [47]:
push!(df_cities, Dict(:population => 122324, :city => "Berkeley"))
df_cities

,city,population
,String,Int64
1,Corvallis,57961
2,Portland,647805
3,Eugene,168916
4,Bend,94520
5,Berkeley,122324


## add columns to the `DataFrame`

In [49]:
df_cities[!, :state] = vcat(["Oregon" for i = 1:4], ["California"])
df_cities

,city,population,state
,String,Int64,String
1,Corvallis,57961,Oregon
2,Portland,647805,Oregon
3,Eugene,168916,Oregon
4,Bend,94520,Oregon
5,Berkeley,122324,California


## how many (rows, columns)?

In [50]:
size(df_cities) # size(df_cities)[1] = # rows, size(df_cities)[2] = # cols 

(5, 3)

## what are the column names?
note that the column names are always symbols! if you have a fancy `String` you want to convert into a `Symbol`, simply e.g. `Symbol("avg salary (USD)")` will convert the `String` `"avg salary (USD)"` to a `Symbol`.

In [51]:
names(df_cities) # will be very handy when there are *many* columns

3-element Array{Symbol,1}:
 :city      
 :population
 :state     

## iterate through the rows of a `DataFrame`

In [61]:
for row in eachrow(df_cities)
    # inside the loop, we have a row (representing a data point)
    # and we can call whatever attribute of that data point we want
    @printf("population of %s, %s is %d\n", row[:city], row[:state], row[:population])
end

population of Corvallis, Oregon is 57961
population of Portland, Oregon is 647805
population of Eugene, Oregon is 168916
population of Bend, Oregon is 94520
population of Berkeley, California is 122324


## iterate through groups in a `DataFrame`
the `groupby` command (common in many languages) partitions the rows in the `DataFrame` into multiple `DataFrame`s such that the rows in each `DataFrame` share a common attribute.

for example, we aim to group the cities by state

In [73]:
gb_state = groupby(df_cities, :state)

,city,population,state
,String,Int64,String
1,Portland,647805,Oregon
2,Eugene,168916,Oregon
3,Bend,94520,Oregon
4,Corvallis,57961,Oregon
,city,population,state
,String,Int64,String
1,Berkeley,122324,California


In [75]:
for df_by_state in gb_state
    this_state = df_by_state[1, :state]
    nb_cities_in_this_state = size(df_by_state)[1]
    
    @printf("%s has %d cities in this dataset.\n", this_state, nb_cities_in_this_state)
end

Oregon has 4 cities in this dataset.
California has 1 cities in this dataset.


## retreive a column from a dataframe
the columns are `Array`s!

In [101]:
# (1) treating the DataFrame like a multi-dimensional array
df_cities[!, :population] # ! = all the rows

5-element Array{Int64,1}:
 647805
 168916
 122324
  94520
  57961

In [102]:
# (2) treating the DataFrame like a `Struct`
df_cities.population

5-element Array{Int64,1}:
 647805
 168916
 122324
  94520
  57961

## retreive a row

In [77]:
df_cities[2, :]

,city,population,state
,String,Int64,String
2,Eugene,168916,Oregon


## retreive a combination of rows/columns

like an array, we can retreive, say, the first 3 rows and only the `:population` and `:city` columns

In [78]:
df_cities[1:3, [:population, :city]]

,population,city
,Int64,String
1,647805,Portland
2,168916,Eugene
3,122324,Berkeley


so dataframes behave like arrays except their columns are labeled and queried by a `Symbol`

## querying a dataframe

all the rows where the city is listed as "Corvallis"

In [79]:
df_cities[df_cities[!, :city] .== "Corvallis", :] # like array slicing with array of bits

,city,population,state
,String,Int64,String
1,Corvallis,57961,Oregon


all the rows where the population is less than 500,000

In [80]:
df_cities[df_cities[!, :population] .< 500000, :] # like array slicing with array of bits

,city,population,state
,String,Int64,String
1,Eugene,168916,Oregon
2,Berkeley,122324,California
3,Bend,94520,Oregon
4,Corvallis,57961,Oregon


all the rows where cities are in the state of Oregon

... this time let's use the `filter` function, where we first pass a function that operates on a row of the data frame and is true or false, true iff we want to keep the row

there is also a `filter!` function that will remove these rows from the dataframe instead of returning a copy with the rows removed

In [81]:
filter(row -> row[:state] == "Oregon", df_cities)

,city,population,state
,String,Int64,String
1,Portland,647805,Oregon
2,Eugene,168916,Oregon
3,Bend,94520,Oregon
4,Corvallis,57961,Oregon


## sorting a dataframe

e.g. permute rows so that cities are listed by population in reverse order

In [82]:
sort!(df_cities, :population, rev=true) # ! means the input dataframe was modified. now the rows are re-ordered
df_cities

,city,population,state
,String,Int64,String
1,Portland,647805,Oregon
2,Eugene,168916,Oregon
3,Berkeley,122324,California
4,Bend,94520,Oregon
5,Corvallis,57961,Oregon


## read a CSV from a file

CSV = comma-separated values, a very common format for storing tables of data

In [87]:
run(`cat incomes.csv`) # running shell command from Julia! this is what the file looks like

city,per_capita_income
Corvallis,19317
Berkeley,38896
San Franscisco,52261


Process(`cat incomes.csv`, ProcessExited(0))

In [88]:
df_incomes = CSV.read("incomes.csv")

,city,per_capita_income
,String,Int64
1,Corvallis,19317
2,Berkeley,38896
3,San Franscisco,52261


## joins
combine two dataframes that have different columns

there are [seven types of joins](http://juliadata.github.io/DataFrames.jl/stable/man/joins/). let's illustrate two here.

goal: join information about *cities* from `df_cities` and `df_incomes`. thus the *key* here is `:city` since we aim to combine rows of the two `DataFrames` on the basis of the `:city` column.

subtlety here: 
* San Francisco is in `df_incomes` but missing from `df_cities`
* Bend, Eugene, Portland are in `df_cities` but missing from `df_incomes`

### inner join
only keep rows where the city is common between the two `DataFrames`
(throw out rows that aren't common between the two)

In [89]:
df = join(df_cities, df_incomes, on=:city, kind=:inner)

,city,population,state,per_capita_income
,String,Int64,String,Int64
1,Berkeley,122324,California,38896
2,Corvallis,57961,Oregon,19317


### outer join
keep all rows, fill with `missing` values when an attribute is missing in either `DataFrame`
(keep rows that aren't common between the two)

In [91]:
df = join(df_cities, df_incomes, on=:city, kind=:outer)

,city,population,state,per_capita_income
,String,Int64⍰,String⍰,Int64⍰
1,Portland,647805,Oregon,missing
2,Eugene,168916,Oregon,missing
3,Berkeley,122324,California,38896
4,Bend,94520,Oregon,missing
5,Corvallis,57961,Oregon,19317
6,San Franscisco,missing,missing,52261


## missing values
Julia has a data type to efficiently handle missing values

In [ ]:
typeof(missing)

columns with missing values are arrays of whatever type but `Union`'d with the `Missing` type

In [95]:
df[!, :population]

6-element Array{Union{Missing, Int64},1}:
 647805       
 168916       
 122324       
  94520       
  57961       
       missing

remove all rows that have a missing attribute

In [97]:
dropmissing(df) # returns a copy, no ! so does not remove the rows from the original df

,city,population,state,per_capita_income
,String,Int64,String,Int64
1,Berkeley,122324,California,38896
2,Corvallis,57961,Oregon,19317


remove all rows with a missing per capita income

In [98]:
dropmissing(df, :per_capita_income)

,city,population,state,per_capita_income
,String,Int64⍰,String⍰,Int64
1,Berkeley,122324,California,38896
2,Corvallis,57961,Oregon,19317
3,San Franscisco,missing,missing,52261


## write `DataFrame` to CSV

In [103]:
CSV.write("joined_df.csv", df)

run(`cat joined_df.csv`)

city,population,state,per_capita_income
Portland,647805,Oregon,
Eugene,168916,Oregon,
Berkeley,122324,California,38896
Bend,94520,Oregon,
Corvallis,57961,Oregon,19317
San Franscisco,,,52261


Process(`cat joined_df.csv`, ProcessExited(0))